In [1]:
#Hi, Welcome to the Project1 : Financial Transactions Dataset: Analytics
print("Financial Transactions Dataset: Analytics")

StatementMeta(, a32129fa-5b8b-427f-a9b8-0d65d191327e, 3, Finished, Available, Finished)

Financial Transactions Dataset: Analytics


In [2]:
#Making the Dataframes from CSV files in Lakehouse
transactionsdf  = spark.read.csv("abfss://Statistic_Analysis@onelake.dfs.fabric.microsoft.com/Statistic_Analysis.Lakehouse/Files/transactions_data.csv",header = True,inferSchema = True)
cardsdf = spark.read.csv("abfss://Statistic_Analysis@onelake.dfs.fabric.microsoft.com/Statistic_Analysis.Lakehouse/Files/cards_data.csv",header=True,inferSchema=True)
clientdf = spark.read.csv("abfss://Statistic_Analysis@onelake.dfs.fabric.microsoft.com/Statistic_Analysis.Lakehouse/Files/users_data.csv",header=True,inferSchema=True)

StatementMeta(, a32129fa-5b8b-427f-a9b8-0d65d191327e, 4, Finished, Available, Finished)

In [3]:
# Try reading the json file with the multiline to handle json object data
Mcodedf = spark.read.option("multiLine", "true").json('abfss://Statistic_Analysis@onelake.dfs.fabric.microsoft.com/Statistic_Analysis.Lakehouse/Files/mcc_codes.json')
#display(Mcodedf) ,-- Mcodes are taken as Column names and Description as Rows

# Approach - Extract Column(Mcodes) and Description(Row) in Dataframes
columns = Mcodedf.columns
first_row = Mcodedf.head()
descriptions = [first_row[col] for col in columns]
# Create a DataFrame with MCCcode and description
Mcodedf = spark.createDataFrame(zip(columns, descriptions), ["MCCcode", "Description"])
Mcodedf.show()

StatementMeta(, a32129fa-5b8b-427f-a9b8-0d65d191327e, 5, Finished, Available, Finished)

+-------+--------------------+
|MCCcode|         Description|
+-------+--------------------+
|   1711|Heating, Plumbing...|
|   3000|          Steelworks|
|   3001|Steel Products Ma...|
|   3005|Miscellaneous Met...|
|   3006|Miscellaneous Fab...|
|   3007|Coated and Lamina...|
|   3008|Steel Drums and B...|
|   3009|Fabricated Struct...|
|   3058|Tools, Parts, Sup...|
|   3066|Miscellaneous Metals|
|   3075|Bolt, Nut, Screw,...|
|   3132|       Leather Goods|
|   3144|Floor Covering St...|
|   3174|Upholstery and Dr...|
|   3256|Brick, Stone, and...|
|   3260|Pottery and Ceramics|
|   3359|Non-Ferrous Metal...|
|   3387|Electroplating, P...|
|   3389|Non-Precious Meta...|
|   3390|Miscellaneous Met...|
+-------+--------------------+
only showing top 20 rows



### **Exploratory Data Analysis**

###### **TRANSACTIONS DATA** - transactiondf DataFrame

In [4]:
# this step will print the Schema of the dataframe
transactionsdf.printSchema()
# this step can be used to check for the datatypes of columns of the dataframe
transactionsdf.dtypes
# Find the Number of Null values in the dataframe and replace with suitable replacements if necessary
from pyspark.sql.functions import col, sum 
# Create a new DataFrame where each column indicates 1 for null and 0 for non-null
null_counts = transactionsdf.select(
    [col(c).isNull().cast("int").alias(c) for c in transactionsdf.columns]
)
# Sum the null values in each column
transactionnullcount = null_counts.agg(
    *[sum(col(c)).alias(c) for c in transactionsdf.columns]
)
transactionnullcount.show()


StatementMeta(, a32129fa-5b8b-427f-a9b8-0d65d191327e, 6, Submitted, Running, Running)

root
 |-- id: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- client_id: integer (nullable = true)
 |-- card_id: integer (nullable = true)
 |-- amount: string (nullable = true)
 |-- use_chip: string (nullable = true)
 |-- merchant_id: integer (nullable = true)
 |-- merchant_city: string (nullable = true)
 |-- merchant_state: string (nullable = true)
 |-- zip: double (nullable = true)
 |-- mcc: integer (nullable = true)
 |-- errors: string (nullable = true)

+---+----+---------+-------+------+--------+-----------+-------------+--------------+-------+---+--------+
| id|date|client_id|card_id|amount|use_chip|merchant_id|merchant_city|merchant_state|    zip|mcc|  errors|
+---+----+---------+-------+------+--------+-----------+-------------+--------------+-------+---+--------+
|  0|   0|        0|      0|     0|       0|          0|            0|       1563700|1652706|  0|13094522|
+---+----+---------+-------+------+--------+-----------+-------------+--------------+---

In [5]:
# Generating the count of unique values for merchant_city and merchant_state
CityCount = transactionsdf.select("merchant_city").distinct().count()
StateCount = transactionsdf.select("merchant_state").distinct().count()
print(f"Different Cities: {CityCount} || Different States: {StateCount}")

#Adding New Column having both City-State in transactionsDF 
from pyspark.sql.functions import col, concat_ws
transactionsdf = transactionsdf.withColumn("CityState", concat_ws(", ", col("merchant_city"), col("merchant_state")))
transactionsdf.select("merchant_city", "merchant_state", "CityState").show()

StatementMeta(, , , Waiting, , Waiting)

Different Cities: 12492 || Different States: 200
+---------------+--------------+-------------------+
|  merchant_city|merchant_state|          CityState|
+---------------+--------------+-------------------+
|         Beulah|            ND|         Beulah, ND|
|     Bettendorf|            IA|     Bettendorf, IA|
|          Vista|            CA|          Vista, CA|
|    Crown Point|            IN|    Crown Point, IN|
|        Harwood|            MD|        Harwood, MD|
|          Bronx|            NY|          Bronx, NY|
|         Beulah|            ND|         Beulah, ND|
|         ONLINE|          NULL|             ONLINE|
|         ONLINE|          NULL|             ONLINE|
|       Flushing|            NY|       Flushing, NY|
|       Pearland|            TX|       Pearland, TX|
|       Brooklyn|            NY|       Brooklyn, NY|
|         Beulah|            ND|         Beulah, ND|
|        Kahului|            HI|        Kahului, HI|
|North Hollywood|            CA|North Hollywood, C

1. Handling Missing Values : Null Values/Error in values around the columns of dataframes
2. Ensuring the Datatypes : Convert the column into authentic datatypes like from string (if it contains symbols) to numeric, and dates in a proper timestamp format.

In [6]:
from pyspark.sql.functions import col, regexp_replace,to_date
from pyspark.sql.types import DoubleType

#Transactiondf have amount column which contains Dollar sign $, cleaning it and converting to double Type by creating a function
def CleanAmount(amount):
    # Remove dollar signs, commas and keep the negative sign intact
    cleaned_amount = regexp_replace(amount, r"[\$,]", "")  # Remove $, ,
    return cleaned_amount.cast("double")
# Apply the function to the 'amount' column and create a new cleaned 'amount' column
transactionsdf = transactionsdf.withColumn("amount", CleanAmount(col("amount")))
transactionsdf = transactionsdf.withColumn("date", to_date(col("date"), 'yyyy-MM-dd HH:mm:ss'))
transactionsdf.show(5)  #transactionsdf.dtypes this will check the datatypes of the columns in TransactionDf

StatementMeta(, , , Waiting, , Waiting)

+-------+----------+---------+-------+------+-----------------+-----------+-------------+--------------+-------+----+------+---------------+
|     id|      date|client_id|card_id|amount|         use_chip|merchant_id|merchant_city|merchant_state|    zip| mcc|errors|      CityState|
+-------+----------+---------+-------+------+-----------------+-----------+-------------+--------------+-------+----+------+---------------+
|7475327|2010-01-01|     1556|   2972| -77.0|Swipe Transaction|      59935|       Beulah|            ND|58523.0|5499|  NULL|     Beulah, ND|
|7475328|2010-01-01|      561|   4575| 14.57|Swipe Transaction|      67570|   Bettendorf|            IA|52722.0|5311|  NULL| Bettendorf, IA|
|7475329|2010-01-01|     1129|    102|  80.0|Swipe Transaction|      27092|        Vista|            CA|92084.0|4829|  NULL|      Vista, CA|
|7475331|2010-01-01|      430|   2860| 200.0|Swipe Transaction|      27092|  Crown Point|            IN|46307.0|4829|  NULL|Crown Point, IN|
|7475332|2010

###### **CARDS DATA** - cardsdf DataFrame

In [7]:
# this step will print the Schema of the dataframe
cardsdf.printSchema()
# this step can be used to check for the datatypes of columns of the dataframe
cardsdf.dtypes
# Find the Number of Null values in the dataframe and replace with suitable replacements if necessary
from pyspark.sql.functions import col, sum 
# Create a new DataFrame where each column indicates 1 for null and 0 for non-null
null_counts = cardsdf.select(
    [col(c).isNull().cast("int").alias(c) for c in cardsdf.columns]
)
# Sum the null values in each column
cardnullcount = null_counts.agg(
    *[sum(col(c)).alias(c) for c in cardsdf.columns]
)
cardnullcount.show()

StatementMeta(, , , Waiting, , Waiting)

root
 |-- id: integer (nullable = true)
 |-- client_id: integer (nullable = true)
 |-- card_brand: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- card_number: long (nullable = true)
 |-- expires: string (nullable = true)
 |-- cvv: integer (nullable = true)
 |-- has_chip: string (nullable = true)
 |-- num_cards_issued: integer (nullable = true)
 |-- credit_limit: string (nullable = true)
 |-- acct_open_date: string (nullable = true)
 |-- year_pin_last_changed: integer (nullable = true)
 |-- card_on_dark_web: string (nullable = true)

+---+---------+----------+---------+-----------+-------+---+--------+----------------+------------+--------------+---------------------+----------------+
| id|client_id|card_brand|card_type|card_number|expires|cvv|has_chip|num_cards_issued|credit_limit|acct_open_date|year_pin_last_changed|card_on_dark_web|
+---+---------+----------+---------+-----------+-------+---+--------+----------------+------------+--------------+---------------

In [8]:
#Converting the Credit Limit of the Card to Double using the function made above in transactiondf
cardsdf = cardsdf.withColumn("credit_limit", CleanAmount(col("credit_limit")))
cardsdf.dtypes

StatementMeta(, , , Waiting, , Waiting)

[('id', 'int'),
 ('client_id', 'int'),
 ('card_brand', 'string'),
 ('card_type', 'string'),
 ('card_number', 'bigint'),
 ('expires', 'string'),
 ('cvv', 'int'),
 ('has_chip', 'string'),
 ('num_cards_issued', 'int'),
 ('credit_limit', 'double'),
 ('acct_open_date', 'string'),
 ('year_pin_last_changed', 'int'),
 ('card_on_dark_web', 'string')]

In [9]:
# Generating the count of unique values for card_brand and card_type
BrandCount = cardsdf.select("card_brand").distinct().count()
TypeCount = cardsdf.select("card_type").distinct().count()
print(f"Different Brands: {BrandCount} || Different Types: {TypeCount}")
# Check if card_on_dark_web column is skewed
cardsdf.groupBy("card_on_dark_web").count().withColumnRenamed("card_on_dark_web","Skewed").show()

StatementMeta(, , , Waiting, , Waiting)

Different Brands: 4 || Different Types: 3
+------+-----+
|Skewed|count|
+------+-----+
|    No| 6146|
+------+-----+



###### **CLIENT DATA** - clientdf DataFrame

In [10]:
# this step will print the Schema of the dataframe
clientdf.printSchema()
# this step can be used to check for the datatypes of columns of the dataframe
clientdf.dtypes
# Find the Number of Null values in the dataframe and replace with suitable replacements if necessary
from pyspark.sql.functions import col, sum 
# Create a new DataFrame where each column indicates 1 for null and 0 for non-null
null_counts = clientdf.select(
    [col(c).isNull().cast("int").alias(c) for c in clientdf.columns]
)
# Sum the null values in each column
clientnullcount = null_counts.agg(
    *[sum(col(c)).alias(c) for c in clientdf.columns]
)
clientnullcount.show()

StatementMeta(, , , Waiting, , Waiting)

root
 |-- id: integer (nullable = true)
 |-- current_age: integer (nullable = true)
 |-- retirement_age: integer (nullable = true)
 |-- birth_year: integer (nullable = true)
 |-- birth_month: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- address: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- per_capita_income: string (nullable = true)
 |-- yearly_income: string (nullable = true)
 |-- total_debt: string (nullable = true)
 |-- credit_score: integer (nullable = true)
 |-- num_credit_cards: integer (nullable = true)

+---+-----------+--------------+----------+-----------+------+-------+--------+---------+-----------------+-------------+----------+------------+----------------+
| id|current_age|retirement_age|birth_year|birth_month|gender|address|latitude|longitude|per_capita_income|yearly_income|total_debt|credit_score|num_credit_cards|
+---+-----------+--------------+----------+-----------+------+-----

In [11]:
#Converting datatypes of Income Columns : {per_capita_income,yearly_income} using the CleanAmount function 
clientdf = clientdf.withColumn("per_capita_income", CleanAmount(col("per_capita_income")))
clientdf = clientdf.withColumn("yearly_income", CleanAmount(col("yearly_income")))
clientdf = clientdf.withColumn("total_debt", CleanAmount(col("total_debt")))
clientdf.dtypes

StatementMeta(, , , Waiting, , Waiting)

[('id', 'int'),
 ('current_age', 'int'),
 ('retirement_age', 'int'),
 ('birth_year', 'int'),
 ('birth_month', 'int'),
 ('gender', 'string'),
 ('address', 'string'),
 ('latitude', 'double'),
 ('longitude', 'double'),
 ('per_capita_income', 'double'),
 ('yearly_income', 'double'),
 ('total_debt', 'double'),
 ('credit_score', 'int'),
 ('num_credit_cards', 'int')]

In [12]:
# Exploring the Distribution of Key Metrics: current_age, retirement_age, gender, income, credit_score
clientdf.select("current_age").describe().show()
clientdf.select("retirement_age").describe().show()
clientdf.groupBy("gender").count().show()
clientdf.select("per_capita_income").describe().show()
clientdf.select("yearly_income").describe().show()
clientdf.select("credit_score").describe().show()

StatementMeta(, , , Waiting, , Waiting)

+-------+------------------+
|summary|       current_age|
+-------+------------------+
|  count|              2000|
|   mean|           45.3915|
| stddev|18.414091537014993|
|    min|                18|
|    max|               101|
+-------+------------------+

+-------+-----------------+
|summary|   retirement_age|
+-------+-----------------+
|  count|             2000|
|   mean|          66.2375|
| stddev|3.628867328663937|
|    min|               50|
|    max|               79|
+-------+-----------------+

+------+-----+
|gender|count|
+------+-----+
|Female| 1016|
|  Male|  984|
+------+-----+

+-------+-----------------+
|summary|per_capita_income|
+-------+-----------------+
|  count|             2000|
|   mean|        23141.928|
| stddev|11324.13735766499|
|    min|              0.0|
|    max|         163145.0|
+-------+-----------------+

+-------+-----------------+
|summary|    yearly_income|
+-------+-----------------+
|  count|             2000|
|   mean|        45715.882|
|

In [13]:
# Check if MCCcode and Description are valid
Mcodedf.printSchema()

# Count of unique MCC codes
Mcodedf.select("MCCcode").distinct().count()

# Distribution of MCC codes
Mcodedf.groupBy("Description").count().orderBy("count", ascending=False).show()


StatementMeta(, , , Waiting, , Waiting)

root
 |-- MCCcode: string (nullable = true)
 |-- Description: string (nullable = true)

+--------------------+-----+
|         Description|count|
+--------------------+-----+
|  Passenger Railways|    2|
|Floor Covering St...|    1|
|Heating, Plumbing...|    1|
|Steel Drums and B...|    1|
|Tools, Parts, Sup...|    1|
|Miscellaneous Metals|    1|
|       Leather Goods|    1|
|Miscellaneous Fab...|    1|
|Steel Products Ma...|    1|
|Miscellaneous Met...|    1|
|Fabricated Struct...|    1|
|Coated and Lamina...|    1|
|          Steelworks|    1|
|Bolt, Nut, Screw,...|    1|
|Industrial Equipm...|    1|
|            Ironwork|    1|
|Non-Ferrous Metal...|    1|
|Non-Precious Meta...|    1|
|Heat Treating Met...|    1|
|Miscellaneous Met...|    1|
+--------------------+-----+
only showing top 20 rows



###### **Writing DataFrames to Delta Tables in Lakehouse**

Saving the DataFrames (transactionsdf, cardsdf, clientdf, mcodedf) as Delta tables in the Lakehouse. The primary goal is to make the data easily accessible for analysis using SQL. By writing the DataFrames as Delta tables, we can leverage SQL queries for further exploration, aggregations, and advanced analytics within the Lakehouse environment. This enables us to use a familiar and efficient method of interacting with the data, optimizing it for both batch and real-time analytics.

In [14]:
'''
# Define the base path where the tables will be stored
base_path = "abfss://Statistic_Analysis@onelake.dfs.fabric.microsoft.com/Statistic_Analysis.Lakehouse/Tables/"

# Write transactionsdf as a Delta table
transactionsdf.write.format("delta").mode("overwrite").save(base_path + "transactions_table")

# Write cardsdf as a Delta table
cardsdf.write.format("delta").mode("overwrite").save(base_path + "cards_table")

# Write clientdf as a Delta table
clientdf.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(base_path + "client_table")

# Write mcodedf as a Delta table
Mcodedf.write.format("delta").mode("overwrite").save(base_path + "mcc_codes_table")
'''


StatementMeta(, , , Waiting, , Waiting)

'\n# Define the base path where the tables will be stored\nbase_path = "abfss://Statistic_Analysis@onelake.dfs.fabric.microsoft.com/Statistic_Analysis.Lakehouse/Tables/"\n\n# Write transactionsdf as a Delta table\ntransactionsdf.write.format("delta").mode("overwrite").save(base_path + "transactions_table")\n\n# Write cardsdf as a Delta table\ncardsdf.write.format("delta").mode("overwrite").save(base_path + "cards_table")\n\n# Write clientdf as a Delta table\nclientdf.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(base_path + "client_table")\n\n# Write mcodedf as a Delta table\nMcodedf.write.format("delta").mode("overwrite").save(base_path + "mcc_codes_table")\n'

 ### **Transaction Data Analysis**

###### **Year-over-Year Transaction Trends**: This analysis tracks the year-over-year (YOY) change in transaction counts, helping to identify significant increases or decreases in transaction volumes between consecutive years. By calculating the percentage change and comparing current year data with the previous year using the lag function, businesses can spot trends and plan strategies accordingly. The analysis highlights the largest increase and decrease in transaction counts, providing insights for future planning and identifying periods of growth or decline.

In [15]:
from pyspark.sql.functions import year, lag, col
from pyspark.sql.window import Window

transactions_per_year = transactionsdf.groupBy(year("date").alias("year")).count().orderBy("year")
# Calculate percentage change in TransactionCounts YOY(YearOverYear)
window_spec = Window.orderBy("year")

#Adding Column for PreviousYearTransactionCount using Lag function
transactions_per_year = transactions_per_year.withColumn(
    "previous_year_count", lag("count", 1).over(window_spec)
)

#Adding Column indicating Percentage Difference of TransactionCount of CurrentYear and PreviousYear
transactions_per_year = transactions_per_year.withColumn(
    "percentage_change", ((col("count") - col("previous_year_count")) / col("previous_year_count")) * 100
)
transactions_per_year.show()

transactions_per_year_filtered = transactions_per_year.filter(col("previous_year_count").isNotNull())

# Finding Trends of TransactionCounts over Years
max_increase = transactions_per_year_filtered.orderBy(col("percentage_change").desc()).first()
max_decrease = transactions_per_year_filtered.orderBy(col("percentage_change").asc()).first()

# Print the Insights from above Data
print(f"Largest positive increase in transactions: Year {max_increase['year']} with {max_increase['percentage_change']}% increase")
print(f"Largest negative decrease in transactions: Year {max_decrease['year']} with {max_decrease['percentage_change']}% decrease")


StatementMeta(, , , Waiting, , Waiting)

+----+-------+-------------------+--------------------+
|year|  count|previous_year_count|   percentage_change|
+----+-------+-------------------+--------------------+
|2010|1240880|               NULL|                NULL|
|2011|1290770|            1240880|    4.02053381471214|
|2012|1321672|            1290770|   2.394074854544187|
|2013|1352808|            1321672|  2.3558038605644973|
|2014|1365537|            1352808|  0.9409317508471269|
|2015|1388065|            1365537|  1.6497539063386784|
|2016|1392117|            1388065| 0.29191716526243366|
|2017|1399308|            1392117|  0.5165514105495443|
|2018|1394792|            1399308|-0.32273094986950696|
|2019|1159966|            1394792|  -16.83591531927341|
+----+-------+-------------------+--------------------+

Largest positive increase in transactions: Year 2011 with 4.02053381471214% increase
Largest negative decrease in transactions: Year 2019 with -16.83591531927341% decrease


###### **Transaction Trends by Month and Weekday**: This analysis identifies transaction patterns by grouping data into monthly and weekday segments, providing valuable insights into transaction volumes across different time periods. The use of `date_format` function helps in creating `MonthYear` and `weekday_name` columns, allowing us to group and count transactions efficiently. By identifying the months and weekdays with the highest and lowest transaction counts, businesses can optimize strategies for peak times and address low-traffic periods.

In [16]:
from pyspark.sql.functions import dayofweek, date_format, col, when
#use of date_format function and 
# Transaction count per month using MonthYear Column
transactions_per_month = transactionsdf.withColumn("MonthYear", date_format("date", "MMMyyyy")
).groupBy("MonthYear").count().withColumnRenamed("count", "TotalTransactions").orderBy(col("TotalTransactions").desc())

# Transaction count per Day using Weekdays as legends
transactions_per_weekday = transactionsdf.withColumn("weekday_name", date_format("date", "EEEE")
).groupBy("weekday_name").count().withColumnRenamed("count", "TotalTransactions").orderBy(col("TotalTransactions").desc())

'''
transactions_per_month.show()
transactions_per_weekday.show()
'''
# Print MaxCount and MinCoutn of transactions per MonthYear
max_month = transactions_per_month.orderBy(col("TotalTransactions").desc()).first()
min_month = transactions_per_month.orderBy(col("TotalTransactions").asc()).first()
print("Transaction Trends over MonthYear:")
print(f"MonthYEar with Maximum Transactions: {max_month['MonthYear']} with {max_month['TotalTransactions']} transactions")
print(f"MonthYear with Minimum Transactions: {min_month['MonthYear']} with {min_month['TotalTransactions']} transactions")

# Print MaxCount and MinCoutn of transactions per weekday
max_weekday = transactions_per_weekday.orderBy(col("TotalTransactions").desc()).first()
min_weekday = transactions_per_weekday.orderBy(col("TotalTransactions").asc()).first()
print("\nTransaction Trends over WeekDays:")
print(f"Weekday with Maximum Transactions: {max_weekday['weekday_name']} with {max_weekday['TotalTransactions']} transactions")
print(f"Weekday with Minimum Transactions: {min_weekday['weekday_name']} with {min_weekday['TotalTransactions']} transactions")

StatementMeta(, , , Waiting, , Waiting)

Transaction Trends over MonthYear:
MonthYEar with Maximum Transactions: Dec2015 with 119396 transactions
MonthYear with Minimum Transactions: Feb2010 with 93470 transactions

Transaction Trends over WeekDays:
Weekday with Maximum Transactions: Thursday with 1918666 transactions
Weekday with Minimum Transactions: Friday with 1895372 transactions


In [17]:
transactionsdf.describe("amount").show()
# TransactionCounts for Debit and Credit Cards
transactionsdf.withColumn("Amount_Type", when(col("amount") > 0, "Credit").otherwise("Debit")
).groupBy("Amount_Type").count().withColumnRenamed('count','TransactionCount').show()

StatementMeta(, , , Waiting, , Waiting)

+-------+-----------------+
|summary|           amount|
+-------+-----------------+
|  count|         13305915|
|   mean|42.97603902324677|
| stddev|81.65574765375865|
|    min|           -500.0|
|    max|           6820.2|
+-------+-----------------+

+-----------+----------------+
|Amount_Type|TransactionCount|
+-----------+----------------+
|     Credit|        12635227|
|      Debit|          670688|
+-----------+----------------+



In [18]:
# We can take Certain other factors in it as well Like the State and City in this
transactionsdf.withColumn(
    "Amount_Type", when(col("amount") > 0, "Credit").otherwise("Debit")
).withColumn(
    "CityState", concat_ws(", ", col("merchant_city"), col("merchant_state"))
).groupBy(
    "Amount_Type", "CityState", "zip"
).count().withColumnRenamed("count", "TransactionCount").show()


StatementMeta(, , , Waiting, , Waiting)

+-----------+-----------------+-------+----------------+
|Amount_Type|        CityState|    zip|TransactionCount|
+-----------+-----------------+-------+----------------+
|     Credit|      Concord, NC|28025.0|            6023|
|     Credit|     Glendale, CA|91205.0|            6065|
|     Credit|      Wooster, OH|44691.0|            8240|
|     Credit|        Cocoa, FL|32926.0|            4856|
|      Debit|    Henderson, NV|89002.0|             480|
|     Credit|         Novi, MI|48377.0|            3925|
|     Credit|     Fairdale, KY|40118.0|             152|
|     Credit|     Franklin, TX|77856.0|            4022|
|     Credit|Green Village, NJ| 7935.0|             782|
|     Credit|  Chino Hills, CA|91709.0|            1722|
|     Credit|      Bristol, CT| 6010.0|            1218|
|      Debit|    Brunswick, OH|44212.0|             729|
|     Credit|      Oshkosh, WI|54902.0|             871|
|     Credit|         Avon, CT| 6001.0|             344|
|     Credit|  Marcus Hook, PA|

###### **Online Transactions Analysis**: This analysis identifies and tracks online transactions by categorizing them into Credit and Debit, filtering for transactions where the city is "ONLINE" and the zip code is missing. By grouping the data by CityState and Year, it reveals trends and patterns in online transactions over time. This helps businesses monitor online activity and adjust strategies based on transaction volumes by region and year.

In [19]:
OnlineTransactionsdf = transactionsdf.withColumn("Amount_Type", when(col("amount") > 0, "Credit").otherwise("Debit")
).filter((col("merchant_city") == "ONLINE") & col("zip").isNull())
OnlineTransactionsdf.show()
OnlineTransactionsdf = OnlineTransactionsdf.withColumn("Year",date_format("date","yyyy"))
OnlineTransactionsdf.groupBy("CityState","Year").count().withColumnRenamed('count','TotalTransactions').orderBy(col("Year").desc()).show()

StatementMeta(, , , Waiting, , Waiting)

+-------+----------+---------+-------+------+------------------+-----------+-------------+--------------+----+----+------+---------+-----------+
|     id|      date|client_id|card_id|amount|          use_chip|merchant_id|merchant_city|merchant_state| zip| mcc|errors|CityState|Amount_Type|
+-------+----------+---------+-------+------+------------------+-----------+-------------+--------------+----+----+------+---------+-----------+
|7475335|2010-01-01|     1684|   2140| 26.46|Online Transaction|      39021|       ONLINE|          NULL|NULL|4784|  NULL|   ONLINE|     Credit|
|7475336|2010-01-01|      335|   5131|261.58|Online Transaction|      50292|       ONLINE|          NULL|NULL|7801|  NULL|   ONLINE|     Credit|
|7475346|2010-01-01|      394|   4717| 26.04|Online Transaction|      39021|       ONLINE|          NULL|NULL|4784|  NULL|   ONLINE|     Credit|
|7475353|2010-01-01|      301|   3742| 10.17|Online Transaction|      39021|       ONLINE|          NULL|NULL|4784|  NULL|   ONLIN

### **Fraud Detection & Error Analysis** 

###### **Fraudulent Transaction Analysis**: This analysis identifies merchants with fraudulent transactions by counting the instances and summing the amounts. It helps pinpoint merchants with higher fraud activity, enabling targeted risk management. The insights can be used to prioritize interventions and improve fraud detection systems.

###### **Analyzing Transactions with Errors (Fraud Detection):**

In [20]:
# Count transactions with errors (fraud)
fraudulent_transactions = transactionsdf.filter(col("errors").isNotNull()).count()
total_transactions = transactionsdf.count()
fraud_percentage = (fraudulent_transactions / total_transactions) * 100
print(f"Percentage of fraudulent transactions: {fraud_percentage:.2f}%")

StatementMeta(, , , Waiting, , Waiting)

Percentage of fraudulent transactions: 1.59%


###### **Fraudulent Transactions by Merchant ID** : This below analysis helps identify merchants involved in fraudulent transactions by counting them and summing the amounts. It’s useful for spotting high-risk merchants and focusing fraud prevention efforts where needed.

In [21]:
from pyspark.sql.functions import col, sum, count
# Fraudulent transactions by Merchant ID
fraud_by_merchant_id = transactionsdf.filter(col("errors").isNotNull()).groupBy("client_id") \
    .agg(count("*").alias("TransactionCount"), sum("amount").alias("TransactionSum") 
    ).withColumnRenamed("client_id", "Client").orderBy("TransactionCount", ascending=False)
fraud_by_merchant_id.show()

StatementMeta(, , , Waiting, , Waiting)

+------+----------------+------------------+
|Client|TransactionCount|    TransactionSum|
+------+----------------+------------------+
|   954|            2935|         226724.76|
|  1888|            2109| 90075.82999999999|
|   464|            1196|         127925.28|
|  1098|            1125|          42302.82|
|  1696|             953|          59278.27|
|   425|             847|          55073.41|
|  1424|             822|          12968.02|
|  1382|             821|          23333.85|
|   373|             816|19862.280000000006|
|   114|             796|          29223.93|
|   476|             741|31039.879999999997|
|  1885|             713|49210.490000000005|
|    53|             675|          26174.66|
|  1963|             654|19544.319999999996|
|    96|             617|          66182.73|
|  1340|             611| 74000.26999999999|
|   208|             601|34501.700000000004|
|  1797|             599|          31679.86|
|   154|             593|45622.990000000005|
|  1241|  

###### **Fraudulent Transactions by Zip Code** : This analysis tracks fraud by identifying suspicious transactions in different regions and over time. By grouping data based on zip code and year, it helps detect patterns and regional trends in fraudulent activities. This insight is crucial for businesses to target areas with higher fraud risk and enhance security measures accordingly.

In [22]:
# Fraudulent transactions by Zip Code
fraud_by_zipcode = transactionsdf.filter(col("errors").isNotNull()).withColumn("Year",date_format("date","yyyy")).groupBy("zip", "Year") \
    .agg(
        count("*").alias("TransactionCount"), sum("amount").alias("TransactionSum")
        ).withColumnRenamed('zip','ZipCode').select("Year", "TransactionCount", "TransactionSum", "ZipCode").orderBy("Year", ascending=False)
fraud_by_zipcode.show()

StatementMeta(, , , Waiting, , Waiting)

+----+----------------+------------------+-------+
|Year|TransactionCount|    TransactionSum|ZipCode|
+----+----------------+------------------+-------+
|2019|              16|            735.48|71019.0|
|2019|               1|             59.06|58425.0|
|2019|               2|            859.94|11219.0|
|2019|              13|            532.12|77086.0|
|2019|               2|              24.5|85716.0|
|2019|               5|            145.22|43812.0|
|2019|               1|             45.62| 7646.0|
|2019|               7|            405.36|53149.0|
|2019|               5|            359.95|13031.0|
|2019|               1|              67.0|54801.0|
|2019|              31|           2423.49|39452.0|
|2019|               2|102.46000000000001|23103.0|
|2019|               2|            -81.29|62565.0|
|2019|               2|49.239999999999995|95821.0|
|2019|              15|            188.13|38606.0|
|2019|              11|            377.53|92505.0|
|2019|               2|140.4200

### **Client Data Analysis**

###### **Client Demographics Analysis (Age, Gender, Income)**: This analysis provides a comprehensive view of the client base by examining key demographics like age, gender, and income. The `describe()` function is used to summarize the distribution of client age and income, while the `groupBy` operation helps to analyze gender distribution. Understanding these demographics helps businesses tailor their products, services, and marketing efforts to specific client segments, improving customer engagement and decision-making.

In [23]:
# Age distribution of clients
clientdf.select("current_age").describe().show()

# Income distribution (per capita and yearly)
clientdf.select("per_capita_income", "yearly_income").describe().show()

# Gender distribution
clientdf.groupBy("gender").count().show()


StatementMeta(, , , Waiting, , Waiting)

+-------+------------------+
|summary|       current_age|
+-------+------------------+
|  count|              2000|
|   mean|           45.3915|
| stddev|18.414091537014993|
|    min|                18|
|    max|               101|
+-------+------------------+

+-------+-----------------+-----------------+
|summary|per_capita_income|    yearly_income|
+-------+-----------------+-----------------+
|  count|             2000|             2000|
|   mean|        23141.928|        45715.882|
| stddev|11324.13735766499|22992.61545631198|
|    min|              0.0|              1.0|
|    max|         163145.0|         307018.0|
+-------+-----------------+-----------------+

+------+-----+
|gender|count|
+------+-----+
|Female| 1016|
|  Male|  984|
+------+-----+



**Average Credit Score**: It computes the average credit score of clients using the credit_score column, helping to understand the overall creditworthiness of the customer base.

**Debt-to-Income Ratio**: It calculates the debt-to-income ratio by dividing total_debt by yearly_income, which is an important metric to assess financial health and risk.

These calculations are useful in identifying trends in client financial health and assessing potential risk factors for loans or other financial services.

In [31]:
from pyspark.sql.functions import avg
# Calculate and print average credit score
avg_credit_score = clientdf.agg({"credit_score": "avg"}).collect()[0][0]
print(f"The average credit score of clients is: {avg_credit_score}")
#Making a Joined DataFrame with transactionsdf 
CreditScoredf = transactionsdf.join(clientdf, transactionsdf.client_id == clientdf.id, "inner")
# calculating the average credit score group by ZipCode,
CreditScoredf = CreditScoredf.filter(col("zip").isNotNull()).groupBy("zip").agg(avg("credit_score").alias("AverageCreditScore")).orderBy("zip")
CreditScoredf.show()
# Debt-to-Income ratio
clientdf.withColumn("debt_to_income", col("total_debt") / col("yearly_income")).select("debt_to_income").describe().show()


StatementMeta(, a32129fa-5b8b-427f-a9b8-0d65d191327e, 33, Finished, Available, Finished)

The average credit score of clients is: 709.7345
+------+------------------+
|   zip|AverageCreditScore|
+------+------------------+
|1001.0| 707.5777777777778|
|1002.0| 681.4444444444445|
|1007.0|             681.0|
|1008.0|             655.0|
|1009.0|             749.0|
|1010.0|             683.0|
|1011.0|             689.5|
|1012.0| 697.2413793103449|
|1013.0| 730.2083333333334|
|1020.0|             682.0|
|1022.0|             772.0|
|1027.0|             748.6|
|1028.0| 703.9532710280374|
|1030.0|           700.125|
|1031.0|             503.0|
|1032.0| 691.3333333333334|
|1033.0|           688.875|
|1034.0| 733.6666666666666|
|1035.0|             741.2|
|1036.0| 720.3888888888889|
+------+------------------+
only showing top 20 rows

+-------+------------------+
|summary|    debt_to_income|
+-------+------------------+
|  count|              2000|
|   mean|1.3816930657665631|
| stddev|0.8692247848503719|
|    min|               0.0|
|    max|  4.97864889017704|
+-------+------------